# Access SQL db and output tables

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard libraries
import logging
import sys
import os
import pathlib

# 3rd party libraries
import geopandas as gpd
import dask.dataframe as dd
from dask.distributed import Client
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl

In [3]:
# Enable viewing of logging outputs
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [4]:
# Display settings
sns.set()
%matplotlib inline
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 5

In [5]:
# Establish connection to pudl database
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine,
    freq='AS'
)

### SQL Access

In [6]:
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])

In [7]:
plants = pd.read_sql("plants_eia860", pudl_engine)

### Output Table Access

In [8]:
pudl_out.plants_eia860()

,plant_id_eia,plant_name_eia,balancing_authority_code_eia,balancing_authority_name_eia,city,county,ferc_cogen_status,ferc_exempt_wholesale_generator,ferc_small_power_producer,grid_voltage_kv,grid_voltage_2_kv,grid_voltage_3_kv,iso_rto_code,latitude,longitude,primary_purpose_naics_id,sector_name,sector_id,service_area,state,street_address,zip_code,timezone,report_date,ash_impoundment,ash_impoundment_lined,ash_impoundment_status,datum,energy_storage,ferc_cogen_docket_no,ferc_exempt_wholesale_generator_docket_no,ferc_small_power_producer_docket_no,liquefied_natural_gas_storage,natural_gas_local_distribution_company,natural_gas_storage,natural_gas_pipeline_name_1,natural_gas_pipeline_name_2,natural_gas_pipeline_name_3,nerc_region,net_metering,pipeline_notes,regulatory_status_code,transmission_distribution_owner_id,transmission_distribution_owner_name,transmission_distribution_owner_state,utility_id_eia,water_source,plant_id_pudl,utility_name_eia,utility_id_pudl
0,1,Sand Point,None,None,Sand Point,Aleutians East,False,False,False,0.48,NaN,NaN,None,55.339722,-160.497222,22.0,IPP Non-CHP,2.0,None,AK,100 Power Plant Way,99661,America/Anchorage,2019-01-01,False,False,None,None,False,None,None,None,False,None,False,None,None,None,None,None,None,NR,63560,"TDX Sand Point Generating, LLC",AK,63560,None,15346,"TDX Sand Point Generating, LLC",6409
1,2,Bankhead Dam,SOCO,"Southern Company Services, Inc. - Trans",Northport,Tuscaloosa,False,False,False,115.00,NaN,NaN,None,33.458665,-87.356823,22.0,Electric Utility,1.0,None,AL,19001 Lock 17 Road,35476,America/Chicago,2019-01-01,False,False,None,None,False,None,None,None,None,None,None,None,None,None,SERC,None,None,RE,195,Alabama Power Co,AL,195,Black Warrior River,854,Alabama Power Co,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127828,63940,Central 40,CISO,California Independent System Operator,Newman,Stanislaus,False,False,False,115.00,NaN,NaN,None,37.246000,-121.104000,22.0,IPP Non-CHP,2.0,None,CA,4951 Sullivan Road,95360,America/Los_Angeles,2019-01-01,None,None,None,None,False,None,None,None,None,None,None,None,None,None,WECC,None,None,NR,14328,Pacific Gas & Electric Co.,CA,63606,None,14628,NaN,<NA>
127829,63941,Oasis Alta,CISO,California Independent System Operator,Mojave,Kern,False,False,False,230.00,NaN,NaN,None,35.059166,-118.291944,22.0,IPP Non-CHP,2.0,None,CA,TBD,93501,America/Los_Angeles,2019-01-01,None,None,None,None,False,None,None,None,None,None,None,None,None,None,WECC,None,None,NR,17609,Southern California Edison Co,CA,63477,None,14690,NaN,<NA>
